# Supervised Ridge

In [1]:
import datetime
import pickle
import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
import warnings
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)

In [3]:
# Read in Test/Train Data
X_train = pd.read_parquet('final_output/X_train.parquet')
X_test = pd.read_parquet('final_output/X_test.parquet')
y_train = pd.read_parquet('final_output/y_train.parquet').values
y_test = pd.read_parquet('final_output/y_test.parquet').values

In [4]:
start_time_reg = datetime.datetime.now()
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'alpha': [0.01, 0.05, 0.1, 0, 1, 10, 100, 1000],
    'fit_intercept': [True, False]
}

ridge = Ridge(solver='auto')
lr_search = HalvingGridSearchCV(ridge, param_grid, scoring='neg_mean_absolute_error', cv=kfold, verbose=1, random_state=42)

# Fit to the training data
lr_search.fit(X_train, y_train)
print('The time used for the HalvingGridSearchCV is ', datetime.datetime.now()-start_time_reg)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 238
max_resources_: 2143
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 16
n_resources: 238
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 1
n_candidates: 6
n_resources: 714
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 2
n_candidates: 2
n_resources: 2142
Fitting 5 folds for each of 2 candidates, totalling 10 fits
The time used for the HalvingGridSearchCV is  0:00:00.509550


E:\Languages\Anaconda\envs\siads696\lib\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.04513e-16): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [5]:
lr_best_model = lr_search.best_estimator_

# Getting the MAE scores for each fold
lr_mae_scores = -1 * lr_search.cv_results_['mean_test_score']
lr_mean_mae = np.mean(lr_mae_scores)
lr_std_mae = np.std(lr_mae_scores)

print(f"Best hyperparameters: {lr_search.best_params_}")
print(f"Mean MAE across folds: {lr_mean_mae}")
print(f"Standard deviation of MAE across folds: {lr_std_mae}")

lr_y_pred = lr_best_model.predict(X_test)
lr_r2 = r2_score(y_test, lr_y_pred)
lr_mae = mean_absolute_error(y_test, lr_y_pred)
lr_mse = mean_squared_error(y_test, lr_y_pred)

print(f'Linear Regression R^2:  {lr_r2}')
print(f'Linear Regression MAE:  {lr_mae}')
print(f'Linear Regression RMSE: {np.sqrt(lr_mse)}')

Best hyperparameters: {'alpha': 0.01, 'fit_intercept': True}
Mean MAE across folds: 1.0984429497390344
Standard deviation of MAE across folds: 0.25010703488757735
Linear Regression R^2:  0.6961149207305362
Linear Regression MAE:  0.8494687458523386
Linear Regression RMSE: 1.134781535702546


In [6]:
# Write out the Ridge Model and Search object
with open('final_output/ridge_search.pkl', 'wb') as f:  # All Grid Search Models
  pickle.dump(lr_search, f)
with open('final_output/ridge_model.pkl', 'wb') as f:  # Best Halving Grid Search Model
  pickle.dump(lr_best_model, f)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e75edf0e-32f1-42b8-8a27-9dc0078a206d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>